# Simple TFX Pipeline for Vertex Pipelines


This notebook-based tutorial will create a simple TFX pipeline and run it using
Google Cloud Vertex Pipelines.  This notebook is based on the TFX pipeline
built in
[Simple TFX Pipeline Tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

Google Cloud Vertex Pipelines helps you to automate, monitor, and govern
your ML systems by orchestrating your ML workflow in a serverless manner. You
can define your ML pipelines using Python with TFX, and then execute your
pipelines on Google Cloud. See
[Vertex Pipelines introduction](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)
to learn more about Vertex Pipelines.

## Setup
### Install python packages

We will install required Python packages including TFX and KFP to author ML
pipelines and submit jobs to Vertex Pipelines.

In [ ]:
!pip install --upgrade pip 
!pip uninstall tensorflow tensorflow-tensorboard tensorflow-io tensorflow-cloud -y 
!pip install tensorflow==1.15.5 
!pip uninstall tfx -y 
!pip install --upgrade "tfx[kfp]<2"

# You may see package dependency errors in the output below. You may ignore these to run the cells of this notebook.

#### Restart the runtime

Restart the runtime to ensure the following cells use the updated versions.

You can restart the runtime with following cell:

In [1]:
# docs_infra: no_execute
import sys
import os
if not 'google.colab' in sys.modules:
  # Automatically restart kernel after installs
  import IPython
  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

Check the package versions.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

# You may see a WARNING issued. You can safely ignore this until the TFX and KFP versions are displayed in the cell output.

### Set up variables

We will set up some variables used to customize the pipelines below. Following
information is required:

* GCP Project id. You can find your Project ID in the panel with your lab instructions.
* GCP Region to run pipelines. For more information about the regions that
Vertex Pipelines is available in, see the
[Vertex AI locations guide](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability).
* Google Cloud Storage Bucket to store pipeline outputs.

**Enter required values in the cell below before running it**.


In [2]:
GOOGLE_CLOUD_PROJECT = 'som-k8s'    
GOOGLE_CLOUD_REGION = 'australia-southeast1'     
GCS_BUCKET_NAME = GOOGLE_CLOUD_PROJECT + '-gcs'

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

Set `gcloud` to use your project.

In [3]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

Updated property [core/project].


In [4]:
PIPELINE_NAME = 'penguin-vertex-pipelines'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for input data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# This is the path where your model will be pushed for serving.
SERVING_MODEL_DIR = 'gs://{}/serving_model/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))
print('MODULE_ROOT: {}'.format(MODULE_ROOT))
print('DATA_ROOT: {}'.format(DATA_ROOT))
print('SERVING_MODEL_DIR: {}'.format(SERVING_MODEL_DIR))


PIPELINE_ROOT: gs://som-k8s-gcs/pipeline_root/penguin-vertex-pipelines
MODULE_ROOT: gs://som-k8s-gcs/pipeline_module/penguin-vertex-pipelines
DATA_ROOT: gs://som-k8s-gcs/data/penguin-vertex-pipelines
SERVING_MODEL_DIR: gs://som-k8s-gcs/serving_model/penguin-vertex-pipelines


### Prepare example data
The dataset we are using is the
[Palmer Penguins dataset](https://allisonhorst.github.io/palmerpenguins/articles/intro.html).

There are four numeric features in this dataset:

* culmen_length_mm
* culmen_depth_mm
* flipper_length_mm
* body_mass_g

All features were already normalized
to have range [0,1]. We will build a classification model which predicts the
`species` of penguins.

We need to make our own copy of the dataset. Because TFX ExampleGen reads
inputs from a directory, we need to create a directory and copy dataset to it
on GCS.

In [5]:
!gsutil mb 'gs://'{GCS_BUCKET_NAME}

Creating gs://som-k8s-gcs/...
ServiceException: 409 A Cloud Storage bucket named 'som-k8s-gcs' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [6]:
!gsutil cp gs://download.tensorflow.org/data/palmer_penguins/penguins_processed.csv {DATA_ROOT}/

Copying gs://download.tensorflow.org/data/palmer_penguins/penguins_processed.csv [Content-Type=application/octet-stream]...
/ [1 files][ 25.0 KiB/ 25.0 KiB]                                                
Operation completed over 1 objects/25.0 KiB.                                     


Take a quick look at the CSV file.

In [7]:
!gsutil cat {DATA_ROOT}/penguins_processed.csv | head

species,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
0,0.2545454545454545,0.6666666666666666,0.15254237288135594,0.2916666666666667
0,0.26909090909090905,0.5119047619047618,0.23728813559322035,0.3055555555555556
0,0.29818181818181805,0.5833333333333334,0.3898305084745763,0.1527777777777778
0,0.16727272727272732,0.7380952380952381,0.3559322033898305,0.20833333333333334
0,0.26181818181818167,0.892857142857143,0.3050847457627119,0.2638888888888889
0,0.24727272727272717,0.5595238095238096,0.15254237288135594,0.2569444444444444
0,0.25818181818181823,0.773809523809524,0.3898305084745763,0.5486111111111112
0,0.32727272727272727,0.5357142857142859,0.1694915254237288,0.1388888888888889
0,0.23636363636363636,0.9642857142857142,0.3220338983050847,0.3055555555555556


You should be able to see five values. `species` is one of 0, 1 or 2, and all other features should have values between 0 and 1.

## Create a pipeline

TFX pipelines are defined using Python APIs. We will define a pipeline which
consists of three components:

* CsvExampleGen: Reads in data files and convert them to TFX internal format for further processing. There are multiple ExampleGens for various formats. In this tutorial, we will use CsvExampleGen which takes CSV file input.
* Trainer: Trains an ML model. Trainer component requires a model definition code from users. You can use TensorFlow APIs to specify how to train a model and save it in a _savedmodel format.
* Pusher: Copies the trained model outside of the TFX pipeline. Pusher component can be thought of an deployment process of the trained ML model.

Our pipeline will be almost identical to a basic [TFX pipeline](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

The only difference is that we don't need to set `metadata_connection_config`
which is used to locate
[ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd) database. Because
Vertex Pipelines uses a managed metadata service, users don't need to care
of it, and we don't need to specify the parameter.

Before actually define the pipeline, we need to write a model code for the
Trainer component first.

### Write model code.

We will create a simple DNN model for classification using TensorFlow Keras API. This model training code will be saved to a separate file.

In this tutorial we will use __Generic Trainer__ of TFX which support Keras-based models. You need to write a Python file containing run_fn function, which is the entrypoint for the `Trainer` component.

In [8]:
_trainer_module_file = 'penguin_trainer.py'

In [9]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils


from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _make_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Writing penguin_trainer.py


Copy the module file to GCS which can be accessed from the pipeline components.
Because model training happens on GCP, we need to upload this model definition. 

Otherwise, you might want to build a container image including the module file
and use the image to run the pipeline.

In [10]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

Copying file://penguin_trainer.py [Content-Type=text/x-python]...
- [1 files][  3.8 KiB/  3.8 KiB]                                                
Operation completed over 1 objects/3.8 KiB.                                      


### Write a pipeline definition

We will define a function to create a TFX pipeline.

In [13]:
# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple and
# slightly modified because we don't need `metadata_path` argument.

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     ) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components)

## Run the pipeline on Vertex Pipelines.

TFX provides multiple orchestrators to run your pipeline. In this tutorial we
will use the Vertex Pipelines together with the Kubeflow V2 dag runner.

We need to define a runner to actually run the pipeline. You will compile
your pipeline into our pipeline definition format using TFX APIs.

The generated definition file can be submitted using kfp client.

In [ ]:
PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
# Following function will write the pipeline definition to PIPELINE_DEFINITION_FILE.
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        serving_model_dir=SERVING_MODEL_DIR))

In [ ]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.run(sync=False)

Visit __Vertex AI > Pipelines__ in your Google Cloud Console page to see the progress.

This job will take about 15 minutes in total to complete. Once complete, return to the lab to check your progress.

now deploying the saved model to a endpoint using gcloud command line tool. Note that there is another way as automated deploying the endpoint with kubeflow runner which is in another project

In [53]:
import os
ENDPOINT_NAME='penguins'
os.environ['SERVING_MODEL_DIR']=SERVING_MODEL_DIR
os.environ['ENDPOINT_NAME']=ENDPOINT_NAME
os.environ['REGION']=GOOGLE_CLOUD_REGION
# os.environ['TF_VERSION']='2-11'
os.environ['TF_VERSION']='2-11'# + tf.__version__[2:4]

detele any endpoint if already created (during development

In [36]:
%%bash
ENDPOINT_IDS=$(gcloud ai endpoints list --region=${REGION} --format="value(name)")
if [ -z "$ENDPOINT_IDS" ]; then
  echo "No endpoints found in region ${REGION}."
else
  # Loop through each endpoint ID and delete it
  for ENDPOINT_ID in $ENDPOINT_IDS; do
    echo "Deleting endpoint: $ENDPOINT_ID"
    gcloud ai endpoints delete $ENDPOINT_ID --region=${REGION} --quiet
  done
  echo "All endpoints in region ${REGION} have been deleted."
fi

ENDPOINT_ID=$(gcloud ai endpoints list --region=$REGION \
              --format='value(ENDPOINT_ID)' --filter=display_name=${ENDPOINT_NAME})
echo ENDPOINT_ID=$ENDPOINT_ID


No endpoints found in region australia-southeast1.
ENDPOINT_ID=


Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]


deploy to the endpoint

In [37]:
%%bash
# note TF_VERSION and ENDPOINT_NAME set in 1st cell
# TF_VERSION=2-6
# ENDPOINT_NAME=penguins

TIMESTAMP=$(date +%Y%m%d-%H%M%S)
MODEL_NAME=${ENDPOINT_NAME}-${TIMESTAMP}
echo MODEL_NAME=$MODEL_NAME
EXPORT_PATH=$(gsutil ls ${SERVING_MODEL_DIR} | tail -1)
echo EXPORT_PATH=$EXPORT_PATH
# for MODEL_ID in $(gcloud ai models list --region=$REGION --format='value(MODEL_ID)' --filter=display_name=${MODEL_NAME}); do
#     echo "Deleting existing $MODEL_NAME ... $MODEL_ID "
# done
# create the model endpoint for deploying the model
if [[ $(gcloud ai endpoints list --region=$REGION \
        --format='value(DISPLAY_NAME)' --filter=display_name=${ENDPOINT_NAME}) ]]; then
    echo "Endpoint for $MODEL_NAME already exists"
else
    echo "Creating Endpoint for $MODEL_NAME"
    gcloud ai endpoints create --region=${REGION} --display-name=${ENDPOINT_NAME}
fi

ENDPOINT_ID=$(gcloud ai endpoints list --region=$REGION \
              --format='value(ENDPOINT_ID)' --filter=display_name=${ENDPOINT_NAME})
echo ENDPOINT_ID=$ENDPOINT_ID

# delete any existing models with this name
for MODEL_ID in $(gcloud ai models list --region=$REGION --format='value(MODEL_ID)' --filter=display_name=${MODEL_NAME}); do
    echo "Deleting existing $MODEL_NAME ... $MODEL_ID "
    gcloud ai models delete --region=$REGION $MODEL_ID
done

# create the model using the parameters docker conatiner image and artifact uri
gcloud ai models upload --region=$REGION --display-name=$MODEL_NAME \
     --container-image-uri=us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.${TF_VERSION}:latest \
     --artifact-uri=$EXPORT_PATH
MODEL_ID=$(gcloud ai models list --region=$REGION --format='value(MODEL_ID)' --filter=display_name=${MODEL_NAME})
echo MODEL_ID=$MODEL_ID

echo REGION=$REGION
deploy the model to the endpoint
gcloud ai endpoints deploy-model $ENDPOINT_ID \
  --region=$REGION \
  --model=$MODEL_ID \
  --display-name=$MODEL_NAME \
  --machine-type=e2-standard-2 \
  --min-replica-count=1 \
  --max-replica-count=1 \
  --traffic-split=0=100

MODEL_NAME=penguins-20240618-010232
EXPORT_PATH=gs://som-k8s-gcs/serving_model/penguin-vertex-pipelines/1718669918/
Creating Endpoint for penguins-20240618-010232
ENDPOINT_ID=6895002233411207168
MODEL_ID=2828286954267738112
REGION=australia-southeast1


Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
Waiting for operation [6755237812846460928]...
....................done.
Created Vertex AI endpoint: projects/41931926481/locations/australia-southeast1/endpoints/6895002233411207168.
Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
Waiting for operation [7881137719689084928]...
...................................done.
Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
bash: line 40: deploy: command not found
Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
Waiting for operation [3386545291573329920]...
.................................................................................................................................

createa dummpy file for test

In [38]:
%%writefile example_input.json
{"instances": [
  {"culmen_length_mm":[0.71],"culmen_depth_mm":[0.38],"flipper_length_mm":[0.98],"body_mass_g":[0.78]},
  {"culmen_length_mm":[0.22],"culmen_depth_mm":[0.18],"flipper_length_mm":[0.18],"body_mass_g":[0.38]}
]}

Writing example_input.json


Make a prediction from the model endpoint using vertex ai utiltiy

In [39]:
%%bash
echo $REGION
ENDPOINT_ID=$(gcloud ai endpoints list --region=$REGION \
              --format='value(ENDPOINT_ID)' --filter=display_name=${ENDPOINT_NAME})
echo $ENDPOINT_ID
gcloud ai endpoints predict $ENDPOINT_ID --region=$REGION --json-request=example_input.json

australia-southeast1
6895002233411207168
[[-3.52395582, -2.48878217, 2.94532919], [0.0843113065, -0.834587, -0.397677928]]


Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
Using endpoint [https://australia-southeast1-prediction-aiplatform.googleapis.com/]


Make a prediction from the model endpoint using python client. first need to get the endpoint_id and assign it to a python variable

In [54]:
import subprocess

result = subprocess.run(
    [
        "bash", "-c",
        f"ENDPOINT_ID=$(gcloud ai endpoints list --region={GOOGLE_CLOUD_REGION} --format='value(ENDPOINT_ID)' --filter=display_name={ENDPOINT_NAME}); echo $ENDPOINT_ID"
    ],
    capture_output=True, text=True
)

ENDPOINT_ID = result.stdout.strip()
print(f"The value of ENDPOINT_ID is: {ENDPOINT_ID}")


The value of ENDPOINT_ID is: 6895002233411207168


In [52]:
import numpy as np
# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

# Set data values for the prediction request.
# Our model expects 4 feature inputs and produces 3 output values for each
# species. Note that the output is logit value rather than probabilities.
# See the model code to understand input / output structure.
instances = [{
    'culmen_length_mm':[0.71],
    'culmen_depth_mm':[0.38],
    'flipper_length_mm':[0.98],
    'body_mass_g': [0.78],
}]

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
# Send a prediction request and get response.
response = client.predict(endpoint=endpoint, instances=instances)
print(response.predictions[0])
# Uses argmax to find the index of the maximum value.
print('species:', np.argmax(response.predictions[0]))

[-3.5239563, -2.48878217, 2.94532919]
species: 2


make a prediction with curl request

In [58]:
%%bash
PROJECT=$(gcloud config get-value project)
ENDPOINT_ID=$(gcloud ai endpoints list --region=$REGION \
              --format='value(ENDPOINT_ID)' --filter=display_name=${ENDPOINT_NAME})

curl -X POST \
  -H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @example_input.json \
  "https://${REGION}-aiplatform.googleapis.com/v1/projects/${PROJECT}/locations/${REGION}/endpoints/${ENDPOINT_ID}:predict"

{
  "predictions": [
    [
      -3.52395582,
      -2.48878217,
      2.94532919
    ],
    [
      0.0843113065,
      -0.834587,
      -0.397677928
    ]
  ],
  "deployedModelId": "3528750627951738880",
  "model": "projects/41931926481/locations/australia-southeast1/models/2828286954267738112",
  "modelDisplayName": "penguins-20240618-010232",
  "modelVersionId": "1"
}


Using endpoint [https://australia-southeast1-aiplatform.googleapis.com/]
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   597    0   375  100   222   1476    874 --:--:-- --:--:-- --:--:--  2341
